In [1]:
import numpy as np
import pandas as pd

In [2]:
### 讀取資料 - dimension stone & sheet size
detail = pd.read_csv("Demand.csv")
detail

,Number,Type,Date,Amount
0,CC110016000000,261,2015/8/27,-116.000
1,CC110016000000,261,2015/9/28,-116.000
2,CC110016000000,261,2015/9/28,-70.000
3,CC110016000000,261,2015/11/2,-115.000
4,CC110016000000,261,2016/5/4,-105.800
...,...,...,...,...
34149,CC110250010300,201,2019/1/16,-10.000
34150,CC110250010300,201,2019/1/28,-10.000
34151,CC110250010300,201,2019/2/21,-10.000
34152,CC110250010300,201,2019/3/27,-29.416


In [3]:
### 將number欄位指派給number變數，並以此為根據加入number_type欄位
number = detail["Number"]
detail["number_type"] = [i[:9] for i in number]

In [4]:
### 以Number_type分類
Number_type = detail.groupby("number_type")

In [5]:
### 以Number_type分類後，共有幾類
print("The number of material: ",len(Number_type.size()))

The number of material:  34


In [6]:
Number_type.size().sort_values(ascending = False).head(10)

number_type
CC1101080    7399
CC1102080    5685
CC1102100    3082
CC1102150    2880
CC1101050    2578
CC1101060    2472
CC1102090    1284
CC1102120    1242
CC1102200    1208
CC1101040    1144
dtype: int64

### Group the data by number

In [7]:
number = detail.groupby("Number")

In [8]:
info = number.size()

In [9]:
info.sort_values(ascending = False).to_csv("material.csv")

In [10]:
demand_by_number = info.sort_values(ascending = False).to_frame(name='Count')
demand_by_number.reset_index(inplace=True)

### Dimension_stone

In [11]:
import re
pattern=r'0{5}$'
dimension_stone = detail[~detail["Number"].str.contains(pattern)]
dimension_stone.reset_index(drop=True, inplace=True)

In [12]:
dimension_stone.insert(0, "Number_type", [i[:9] for i in dimension_stone["Number"]])

In [13]:
dimension_stone

,Number_type,Number,Type,Date,Amount,number_type
0,CC1100160,CC110016004950,261,2013/5/11,-169.000,CC1100160
1,CC1100160,CC110016004950,262,2013/5/11,57.000,CC1100160
2,CC1100160,CC110016004950,261,2013/6/4,-57.000,CC1100160
3,CC1100160,CC110016004950,261,2013/6/18,-170.000,CC1100160
4,CC1100160,CC110016004950,201,2014/9/5,-326.200,CC1100160
...,...,...,...,...,...,...
32018,CC1102500,CC110250010300,201,2019/1/16,-10.000,CC1102500
32019,CC1102500,CC110250010300,201,2019/1/28,-10.000,CC1102500
32020,CC1102500,CC110250010300,201,2019/2/21,-10.000,CC1102500
32021,CC1102500,CC110250010300,201,2019/3/27,-29.416,CC1102500


In [14]:
dimension_stone.groupby("Number_type").size().sort_values(ascending = False).to_csv("material_group.csv")

### Transform the data to weekly data

In [15]:
import datetime
 
def dateRange(beginDate, endDate):
    dates = []
    date = beginDate
    
    while date <= endDate:
        dates.append(date)
        date = date + datetime.timedelta(1)
    
    return dates

In [16]:
start = datetime.datetime.strptime('2013/05/01', "%Y/%m/%d")
end = datetime.datetime.strptime('2020/04/30', "%Y/%m/%d")

dates = dateRange(start, end)
dates_data = pd.DataFrame(dates, columns=["Date"])

In [17]:
dimension_stone["Date"] = pd.to_datetime(dimension_stone['Date'], format='%Y/%m/%d')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [18]:
dates_data

,Date
0,2013-05-01
1,2013-05-02
2,2013-05-03
3,2013-05-04
4,2013-05-05
...,...
2552,2020-04-26
2553,2020-04-27
2554,2020-04-28
2555,2020-04-29


In [19]:
dimension_stone.groupby("Number_type").size().sort_values(ascending = False)

Number_type
CC1101080    6990
CC1102080    5364
CC1102100    2838
CC1102150    2773
CC1101050    2414
CC1101060    2260
CC1102200    1208
CC1102090    1169
CC1102120    1127
CC1101040    1037
CC1102070     968
CC1102035     679
CC1102300     502
CC1102400     377
CC1102160     366
CC1101090     342
CC1101100     303
CC1102500     286
CC1102250     283
CC1100160     153
CC1101150     148
CC1102350     113
CC1100250      96
CC1101200      64
CC1102140      50
CC1100200      39
CC1100300      21
CC1102030      19
CC1102050      16
CC1100400      10
CC1101035       4
CC1101070       2
CC1102000       1
CC1102060       1
dtype: int64

In [20]:
Number_types = dimension_stone["Number_type"].unique()

In [21]:
def dataToWeek(data, step= 7):
    
    weekly_df = pd.DataFrame()
    
    weekly_df = weekly_df.append([np.round(np.mean(data.iloc[i:i+step]),2) for i in range(0, len(data), step)])

    return weekly_df

In [22]:
top_ten_consumption = pd.DataFrame(columns=["ID", "Amount"])

for Number_type in Number_types:
    material = dimension_stone[dimension_stone["Number_type"]==Number_type]

    if material.shape[0] >1000:
#         print("The material ID: %s"%material["Number_type"].values[1])
        data = pd.merge(dates_data, material, how='left', on='Date')
        data = data["Amount"]
        weekly_data = dataToWeek(data, step= 7)

#         print(len(weekly_data[~np.isnan(weekly_data)]))

        new = {
            "ID" : material["Number_type"].values[1],
            "Amount" : len(weekly_data[~np.isnan(weekly_data)])
        }

        top_ten_consumption = top_ten_consumption.append(new,ignore_index=True) 

        ##Way I
#         print (sum(np.isnan(weekly_data)==False))

        ##Way II
#         amount = Data["Amount"]
#         print (len(amount[~np.isnan(amount)]))

In [23]:
# top_ten_consumption = data_volume(1000, Number_types)
top_ten_consumption.sort_values(by = 'Amount', ascending=False, ignore_index=True).to_csv("material_top_ten.csv")

### Transform daily data to weekly data

In [26]:
material_weekly = pd.DataFrame(columns=["ID", "Amount"])

for Number_type in Number_types:
    material = dimension_stone[dimension_stone["Number_type"]==Number_type]


    data = pd.merge(dates_data, material, how='left', on='Date')
    data = data["Amount"]
    weekly_data = dataToWeek(data, step= 7)


    new = {
        "ID" : material["Number_type"].values[0],
        "Amount" : int(weekly_data.count())
    }

    material_weekly = material_weekly.append(new,ignore_index=True) 

        ##Way I
#         print (sum(np.isnan(weekly_data)==False))

        ##Way II
#         amount = Data["Amount"]
#         print (len(amount[~np.isnan(amount)]))

In [27]:
material_weekly.sort_values(by = 'Amount', ascending=False, ignore_index=True).to_csv("material_weekly.csv", index=False)